In [28]:
!pip install --upgrade --no-cache-dir gdown

!pip install -q hazm
!pip install -q parsivar

!pip install -q datasets
!pip install -q transformers 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.3 which is incompatible.
parsivar 0.2.3 requires nltk==3.4.5, but you have nltk 3.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.4.5 which is incompatible.
hazm 0.7.0 requires nltk==3.3, but you have nltk 3.4.5 which is incompatible.


In [29]:
!pip install -q "datasets==2.10.1" # previously 2.9.0

In [30]:
import datasets
datasets.__version__

'2.10.1'

In [ ]:
from IPython.display import FileLink

In [ ]:
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [10]:
!gdown 1ZNldbdDACJdu1uB9QVYgiLTKEWJGBzYK # download train_test dataset preprocessed

Downloading...
From: https://drive.google.com/uc?id=1ZNldbdDACJdu1uB9QVYgiLTKEWJGBzYK
To: /kaggle/working/train_test_dataset_torch.zip
100%|█████████████████████████████████████████| 130M/130M [00:01<00:00, 111MB/s]


In [11]:
!unzip train_test_dataset_torch.zip

Archive:  train_test_dataset_torch.zip
   creating: test_dataset/
  inflating: test_dataset/data-00000-of-00001.arrow  
  inflating: test_dataset/dataset_info.json  
  inflating: test_dataset/state.json  
   creating: train_dataset/
  inflating: train_dataset/data-00000-of-00002.arrow  
  inflating: train_dataset/data-00001-of-00002.arrow  
  inflating: train_dataset/dataset_info.json  
  inflating: train_dataset/state.json  


## Imports

In [31]:
import csv
import json
import sqlite3

from collections import defaultdict, Counter

import hazm
from parsivar import Normalizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
from torch import nn

from datasets import Dataset, DatasetDict, load_dataset, load_metric, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer, AutoModel
from transformers import AdamW

## Load data

In [32]:
ls

__notebook_source__.ipynb   torobv3_bert_epoch-1_5th.w
test_dataset/               train_dataset/
torobv3_bert_epoch-1_4th.w  train_test_dataset_torch.zip


In [ ]:
# !unzip /content/drive/MyDrive/train_data_complete.csv.zip

In [ ]:
data_df = pd.read_csv("./train_data_complete.csv", encoding="utf-8")
data_df.head()

## Load Model

In [33]:
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)

## Preprocess Data

In [ ]:
data_df

In [ ]:
data_df.drop(columns=["Unnamed: 0"], inplace=True)


In [ ]:
data_df = data_df.sample(frac=1, random_state=40).reset_index()
data_df


## Normalizer

In [34]:
class MyNormalizer:
    def __init__(self):
        self.parsivar_normalizer = Normalizer(
            statistical_space_correction=True,
            half_space_char=" ",
            pinglish_conversion_needed=True,
        )
        self.hazm_normalizer = hazm.Normalizer(
            remove_extra_spaces=True,
            persian_numbers=True,
            persian_style=True,
            punctuation_spacing=False,
            remove_diacritics=True,
            affix_spacing=False,
            token_based=True,
        )

    def normilize(self, txt):
        return self.hazm_normalizer.normalize(
            self.parsivar_normalizer.normalize(
                txt.replace("\n", " ").replace("\u200c", " ").lower().strip()
            )
        )


In [35]:
normalizer = MyNormalizer()


In [36]:
normalizer.normilize("34.0")


'۳۴ ٫۰'

In [15]:
def preprocess(record):
    query = normalizer.normilize(record["query"])
    p_des = normalizer.normilize(record["p_des"])
    popularity = str(record["popularity"])
    new_words = " ".join(record["new words"])

    encoded_text = tokenizer(
        query,
        popularity + " " + p_des + " " + new_words,
        truncation=True,
        max_length=512,
    )

    label = record["relevance"] * 10

    return {
        "input_ids": encoded_text["input_ids"],
        "attention_mask": encoded_text["attention_mask"],
        "token_type_ids": encoded_text["token_type_ids"],
        "label": label,
    }


In [16]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


NameError: name 'train_df' is not defined

In [17]:
train_dataset, test_dataset

NameError: name 'train_dataset' is not defined

In [ ]:
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess, remove_columns=test_dataset.column_names)

In [ ]:
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [ ]:
train_dataset.save_to_disk("./train_dataset")
test_dataset.save_to_disk("./test_dataset")

# Model

In [37]:
class BaseModel(nn.Module):
    def __init__(self, c_model):
        super(BaseModel, self).__init__()
        self.bert = AutoModel.from_pretrained(c_model)
        self.fc = nn.Linear(self.bert.config.hidden_size, 256)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, 1)
        

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        out = self.fc(output[1])
        return self.fc2(self.relu(out))


## Validation, Train, Test

In [38]:
def validation(model, val_loader, loss_fn, test=True, device="cuda"):
    m = nn.Sigmoid()
    val_loss = 0.0
    model.eval()
    reals = []
    preds_list = []
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"]
        with torch.no_grad():
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
        loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
        val_loss += loss.item()
    val_loss = val_loss / len(val_loader)
    return val_loss


In [39]:
def train(
    model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda"
):
    m = nn.Sigmoid()
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        val_loss = 0.0
        model.train()
        t_labels = []
        t_preds = []
        batch_idx = 0
        for batch in tqdm_notebook(train_loader):
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"]
            t_labels += labels.numpy().tolist()
            optimizer.zero_grad()
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
            # print loss after 1000 batches
            if batch_idx % 1000 == 0:
                print(
                    "Epoch: {}, Batch: {}, Loss: {:.4f}".format(
                    # Val Loss: {:.4f}".format(
                        epoch,
                        batch_idx,
                        training_loss / (batch_idx + 1),
                        # validation(model, test_loader, loss_fn, device=device),
                    )
                )
            batch_idx += 1

        training_loss = training_loss / len(train_loader)
        print("Epoch: {},training loss: {:.4f}  ".format(epoch, training_loss))
    test_loss = validation(model, test_loader, loss_fn, device=device)
    print("test loss: {:.2f} ".format(test_loss))


In [40]:
def collate_batch(batch):
    input_ids = []
    attention_mask = []
    token_type_ids = []
    labels = []
    for b in batch:
        input_ids.append(b["input_ids"])
        token_type_ids.append(b["token_type_ids"])
        attention_mask.append(b["attention_mask"])
        labels.append(b["label"])
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=0.0
    )
    attention_mask = torch.nn.utils.rnn.pad_sequence(
        attention_mask, batch_first=True, padding_value=0.0
    )
    token_type_ids = torch.nn.utils.rnn.pad_sequence(
        token_type_ids, batch_first=True, padding_value=0.0
    )
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
        "label": torch.Tensor(labels).to(torch.long),
    }


## Model Initialization

In [41]:
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaseModel(c_model)
model = nn.DataParallel(model)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 1st cut

In [18]:
train_dataset = Dataset.load_from_disk("./train_dataset")
test_dataset = Dataset.load_from_disk("./test_dataset")
train_data = train_dataset.select(range(1200000, 1500000))

In [19]:
# train_dataset = train_dataset.shuffle()
train_data.set_format(type='torch')
test_dataset.set_format(type='torch')

In [20]:
train_data, test_dataset

(Dataset({
     features: ['label', 'input_ids', 'attention_mask', 'token_type_ids'],
     num_rows: 300000
 }),
 Dataset({
     features: ['label', 'input_ids', 'attention_mask', 'token_type_ids'],
     num_rows: 71361
 }))

In [22]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=32, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, collate_fn=collate_batch, shuffle=True
)

In [23]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

  0%|          | 0/9375 [00:00<?, ?it/s]

Epoch: 0,training loss: 115.7807  
test loss: 117.27 


In [25]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-1_5th" + ".w",
)

In [ ]:
torch.save(
    model.state_dict(), "/kaggle/working/" + "torobv3_bert_epoch-1_1st" + ".w"
)

## 2nd cut

In [ ]:
model.load_state_dict(
    torch.load("/kaggle/working/" + "torobv3_bert_epoch-1_1st" + ".w"),
    strict=True,
)

In [ ]:
train_data = train_dataset.select(range(500000,  700000))
train_data.set_format(type="torch")

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=32, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, collate_fn=collate_batch, shuffle=True
)

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


In [ ]:
torch.save(
    model.state_dict(), "/kaggle/working/" + "torobv3_bert_epoch-1_2nd" + ".w"
)

## 3rd cut

In [ ]:
ls

In [ ]:
model.load_state_dict(
    torch.load("/kaggle/working/" + "torobv3_bert_epoch-1_2nd" + ".w"),
    strict=True,
)

In [ ]:
train_data = train_dataset.select(range(700000, 900000))
train_data.set_format(type="torch")

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, collate_fn=collate_batch, shuffle=True)

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

In [ ]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-1_3rd" + ".w",
)

In [26]:
from IPython.display import FileLink

In [27]:
FileLink(r'torobv3_bert_epoch-1_5th.w')

/kaggle/working/torobv3_bert_epoch-1_5th.w

## 4th cut

In [ ]:
ls

In [ ]:
model.load_state_dict(
    torch.load("/kaggle/working/" + "torobv3_bert_epoch-1_3rd" + ".w"),
    strict=True,
)

In [ ]:
train_data = train_dataset.select(range(900000, 1200000))
train_data.set_format(type="torch")

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data , batch_size=32, collate_fn=collate_batch ,shuffle=True)

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

In [ ]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-1_4th" + ".w",
)

## 5th 250K

In [ ]:
model.load_state_dict(
    torch.load("/kaggle/working/" + "torobv3_bert_epoch-1_4th" + ".w"),
    strict=True,
)

In [ ]:
train_data = train_dataset.select(range(1200000, 1500000 ))
train_data.set_format(type="torch")
train_loader = torch.utils.data.DataLoader(train_data , batch_size=32, collate_fn=collate_batch ,shuffle=True)

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

In [ ]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-1_5th" + ".w",
)

## 6th 250K

In [ ]:
train_df = data_df[1000000:1250000]
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, collate_fn=collate_batch, shuffle=True
)


In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


In [ ]:
torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-1_6th" + ".w",
)


# Second Epoch

## 1st Cut

In [42]:
model.load_state_dict(
    torch.load("/kaggle/working/" + "torobv3_bert_epoch-1_5th" + ".w"),
    strict=True,
)

<All keys matched successfully>

In [43]:
train_dataset = Dataset.load_from_disk("./train_dataset")
test_dataset = Dataset.load_from_disk("./test_dataset")

In [47]:
# train_data = train_dataset.select(range(1200000, 1500000))
train_data = train_dataset.select(range(0, 600000))

In [48]:
train_data.set_format(type='torch')
test_dataset.set_format(type='torch')

In [49]:
train_data, test_dataset

(Dataset({
     features: ['label', 'input_ids', 'attention_mask', 'token_type_ids'],
     num_rows: 600000
 }),
 Dataset({
     features: ['label', 'input_ids', 'attention_mask', 'token_type_ids'],
     num_rows: 71361
 }))

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=32, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, collate_fn=collate_batch, shuffle=True
)

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

In [ ]:

torch.save(
    model.state_dict(),
    "/kaggle/working/" + "torobv3_bert_epoch-2_1st" + ".w",
)

In [ ]:
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaseModel(c_model)
model = nn.DataParallel(model)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()

train_dataset = Dataset.load_from_disk("./train_dataset")
test_dataset = Dataset.load_from_disk("./test_dataset")
test_dataset.set_format(type='torch')

batch_size = 32  # batch size for training
test_batch_size = 1  # batch size for testing
num_samples_per_part = 200_000  # number of samples in each part
max_index = 1_500_000  # maximum index of train data

num_samples = len(train_dataset)
num_parts = (max_index + num_samples_per_part - 1) // num_samples_per_part  # round up

for epoch in range(1, 3):  # set the number of epochs you want to run
    for part in range(num_parts):
        start_idx = part * num_samples_per_part
        end_idx = min((part + 1) * num_samples_per_part, max_index)
        train_data = train_dataset.select(range(start_idx, end_idx))

        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

        model.load_state_dict(torch.load(f"/kaggle/working/torobv3_bert_epoch-{epoch-1}_part-{part}.w"), strict=True)
        model.to(device)

        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

        train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device=device)

        torch.save(model.state_dict(), f"/kaggle/working/torobv3_bert_epoch-{epoch}_part-{part}.w")
